In [5]:
import torch
import utils
from datetime import datetime
import os
import math
from matplotlib import pyplot as plt
from tensorflow.keras.utils import load_img, img_to_array

In [6]:

if not os.path.exists("./yolov5"):
    !git clone https://github.com/ultralytics/yolov5  # clone
    !cd yolov5
    !pip install -qr ./yolov5/requirements.txt  # install
else: 
    print(f"Yolov5 already installed")


#display = utils.notebook_init()  # checks

Yolov5 already installed


### Helper functions

In [7]:
def convert_annotations_DOTA_to_YOLOv5(dota_annotations_path, dota_images_path, yolov5_annotations_path):
    

    # define the DOTA class mapping
    class_mapping = {
        'plane': 0,
        'baseball-diamond': 1,
        'bridge': 2,
        'ground-track-field': 3,
        'small-vehicle': 4,
        'large-vehicle': 5,
        'ship': 6,
        'tennis-court': 7,
        'basketball-court': 8,
        'storage-tank': 9,
        'soccer-ball-field': 10,
        'roundabout': 11,
        'harbor': 12,
        'swimming-pool': 13,
        'helicopter': 14,
        'container-crane': 15
    }


    # loop through each annotation file in the DOTA annotations directory
    for filename in os.listdir(dota_annotations_path):
        if filename.endswith('.txt'):
            # read in the DOTA annotation file
            with open(os.path.join(dota_annotations_path, filename), 'r') as f:
                lines = f.readlines()
            # extract the image dimensions from the corresponding image file
            image_filename = os.path.splitext(filename)[0] + '.png'
            image = cv2.imread(os.path.join(dota_images_path, image_filename))
            height, width, _ = image.shape
            # create a new file for the YOLOv5 annotations
            yolov5_filename = os.path.splitext(filename)[0] + '.txt'
            yolov5_filepath = os.path.join(yolov5_annotations_path, yolov5_filename)
            with open(yolov5_filepath, 'w') as f:
                # loop through each object in the DOTA annotation file
                for line in lines:
                    parts = line.split()
                    class_name = parts[8]
                    class_id = class_mapping[class_name]
                    x1, y1, x2, y2, x3, y3, x4, y4 = map(float, parts[:8])
                    # compute the center coordinates and dimensions of the bounding box
                    x_center = (x1 + x3) / 2 / width
                    y_center = (y1 + y3) / 2 / height
                    bbox_width = abs(x3 - x1) / width
                    bbox_height = abs(y3 - y1) / height
                    # write the YOLOv5 annotation to the output file
                    f.write('{} {:.6f} {:.6f} {:.6f} {:.6f}\n'.format(class_id, x_center, y_center, bbox_width, bbox_height))


In [8]:
def get_images(base_path, first_index=None, last_index=None):
    file_types = [".jpg", ".png", ".jepg"]
    result = []
    
    all_files = os.listdir(base_path)

    if first_index == None:
        first_index = 0
    if last_index == None:
        last_index = len(all_files) - 1

    

    for i,file in enumerate(all_files):
        if any([x in file for x in file_types]):
            if i <= last_index and i >= first_index:
                result.append(os.path.join(base_path,file).replace("\\", "/"))

    return result

In [18]:
def create_image_description_file(items, base_path, name):
    file_path = base_path + "/" + name + ".txt"

    with open(file_path, 'w') as fp:
        print(f"Writing file: {file_path}")
        for item in items:
            # write each item on a new line
            fp.write(".%s\n" % item)
        print(f'Done')

In [10]:
def isImage(filename):
    file_types = ['.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif']
    
    if any([x in filename for x in file_types]):
        return True
    else:
        return False; 

In [11]:
def countObjectsDetected(labels_folder_path):
    num_detections = 0

    label_files = os.listdir(labels_folder_path)
    for i in label_files:
        
        with open(os.path.join(labels_folder_path,i)) as file:
            lines = file.readlines()
            num_detections += len(lines)
            
    #print(f"{num_detections} objects were detected")
    return num_detections

print(countObjectsDetected("yolov5/runs/detect/weights_yolov5s.pt_29-05-2023_174639/labels"))

58


In [12]:
def displayImagesFromFolder(folder_path,
                            num,
                            title=""):
    
        
    base_dir = folder_path
    imgs_to_display = int(num)

    # get all files
    files = os.listdir(base_dir)

    # filter out non-images
    files = [x for x in files if isImage(x)]

    # make full paths
    files = [os.path.join(base_dir, x) for x in files]

    files = files[0 : imgs_to_display]



    cols = 2
    rows = math.ceil(len(files)/2)


    fig, axs = plt.subplots(rows, cols, figsize=(600,300), gridspec_kw={'width_ratios': [1, 1]})

    fig.suptitle(title, fontsize=16)


    for i, img in enumerate(files):
        col = i%2
        row = i//2
        
        if row == rows and col == cols:
            break
        
        axs[row, col].imshow(load_img(img))
        axs[row, col].get_xaxis().set_visible(False)
        axs[row, col].get_yaxis().set_visible(False)
        print(i)
    plt.show()

### Convert the labels from DOTA to YOLOv5 (only required once)

Make sure to have the following file structure: 

 parent<br>
 ├── current file<br> 
 ├── yolov5<br>
 └── datasets<br>
&emsp;&emsp;└── custom_dataset<br>
&emsp;&emsp;&emsp;&emsp;└──  images<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── train<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── test<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── initial_tests<br>
&emsp;&emsp;&emsp;&emsp;└── labels<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── test<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;└── train<br>



In [13]:
convert_train_annotations = False
convert_test_annotations  = False

if convert_test_annotations:
    # convert the annotations for the train folder
    path_to_train_images_DOTA = ""
    path_to_train_labels_DOTA = ""
    target_YOLOv5_annotations_path = ""

    convert_annotations_DOTA_to_YOLOv5(path_to_train_labels_DOTA, path_to_train_images_DOTA, target_YOLOv5_annotations_path)


if convert_train_annotations: 
    # convert the annotations for the test folder
    path_to_test_images_DOTA = r"../datasets/custom_dataset/images/test/"
    path_to_test_labels_DOTA = r"C:/Users/hsack/Downloads/test/aerial txts 1024"
    target_YOLOv5_annotations_path = r"../datasets/custom_dataset/labels/test/"

    convert_annotations_DOTA_to_YOLOv5(path_to_test_labels_DOTA, path_to_test_images_DOTA, target_YOLOv5_annotations_path)

### Create image path files


In [14]:
path_dataset = './datasets/custom_dataset'
path_dataset_images = path_dataset + '/images'

path_test_images = path_dataset_images + '/test'
path_train_images = path_dataset_images + '/train'
path_val_images  = path_dataset_images + '/train'
path_initial_test_images = path_dataset_images + '/initial_tests'


num_images_in_train = len(os.listdir(path_train_images))
num_images_in_test  = len(os.listdir(path_test_images))
num_images_in_initial_test  = len(os.listdir(path_initial_test_images))

print(f"There are {num_images_in_train}\timages in the train\tfolder")
print(f"There are {num_images_in_test}\timages in the test\tfolder")
print(f"There are {num_images_in_initial_test}\timages in the test\tfolder")



There are 1000	images in the train	folder
There are 240	images in the test	folder
There are 31	images in the test	folder


In [19]:
num_train_images = 800
num_val_images   = 200
num_test_images  = 240

train = get_images(path_train_images, 0, num_train_images-1)
create_image_description_file(train, path_dataset, "train")

val = get_images(path_val_images, num_images_in_train-num_val_images, num_images_in_train-1)
create_image_description_file(val,path_dataset, "val")

test = get_images(path_initial_test_images, 0 , 10)
create_image_description_file(test, path_dataset, "test")

initial_test = get_images(path_test_images, 0, 4)
create_image_description_file(initial_test, path_dataset, "initial_test")

Writing file: ./datasets/custom_dataset/train.txt
Done
Writing file: ./datasets/custom_dataset/val.txt
Done
Writing file: ./datasets/custom_dataset/test.txt
Done
Writing file: ./datasets/custom_dataset/initial_test.txt
Done


## Run the (untrained) model on several test-images

In [13]:
# Run the model with the yolov5s weights
weights_folder = ""
weights = "yolov5s.pt"
run_name = "weights_"+ weights + "_" + datetime.now().strftime("%d-%m-%Y_%H%M%S")

path_to_initial_test_folder = "./datasets/custom_dataset/images/initial_tests"

y5s_weights_run = f"./yolov5/runs/detect/{run_name}"

# detect with original yolov5s.pt weights
!python ./yolov5/detect.py --weights {weights} --img 640 --conf 0.25 --source {path_to_initial_test_folder} --name {run_name}  --line-thickness 10 --save-txt
      

/bin/bash: /home/hsack/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
detect: weights=['yolov5s.pt'], source=./datasets/custom_dataset/images/initial_tests, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=weights_yolov5s.pt_29-05-2023_184745, exist_ok=False, line_thickness=10, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.9.16 torch-2.0.1+cu117 CUDA:0 (NVIDIA T600 Laptop GPU, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
image 1/31 /home/hsack/practical_yolov5/datasets/custom_dataset/images/initial_tests/P10747.png: 448x640 2 trucks, 1 umbrella, 33.9ms
image 2/31 /home/hsack/practical_yolov5

In [14]:
# Run the model with the yolov5m weights
weights_folder = ""
weights = "yolov5m.pt"
run_name = "weights_"+ weights + "_" + datetime.now().strftime("%d-%m-%Y_%H%M%S")

weights_path = os.path.join(weights_folder, weights)
path_to_initial_test_folder = "./datasets/custom_dataset/images/initial_tests"

y5m_weights_run = f"./yolov5/runs/detect/{run_name}"

# detect with original yolov5s.pt weights
!python ./yolov5/detect.py --weights {weights} --img 640 --conf 0.25 --source {path_to_initial_test_folder} --name {run_name}  --line-thickness 10 --save-txt
      

/bin/bash: /home/hsack/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
detect: weights=['yolov5m.pt'], source=./datasets/custom_dataset/images/initial_tests, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=weights_yolov5m.pt_29-05-2023_184916, exist_ok=False, line_thickness=10, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.9.16 torch-2.0.1+cu117 CUDA:0 (NVIDIA T600 Laptop GPU, 4096MiB)

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
image 1/31 /home/hsack/practical_yolov5/datasets/custom_dataset/images/initial_tests/P10747.png: 448x640 1 person, 9 cars, 1 bus, 2 trucks, 41.0ms
image 2/31 /home/hsack/pr

In [15]:
# Run the model with the yolov5l weights
weights_folder = ""
weights = "yolov5l.pt"
run_name = "weights_"+ weights + "_" + datetime.now().strftime("%d-%m-%Y_%H%M%S")

path_to_initial_test_folder = "./datasets/custom_dataset/images/initial_tests"

y5l_weights_run = f"./yolov5/runs/detect/{run_name}"

# detect with original yolov5s.pt weights
!python ./yolov5/detect.py --weights {weights} --img 640 --conf 0.25 --source {path_to_initial_test_folder} --name {run_name}  --line-thickness 10 --save-txt
      

/bin/bash: /home/hsack/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
detect: weights=['yolov5l.pt'], source=./datasets/custom_dataset/images/initial_tests, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=weights_yolov5l.pt_29-05-2023_185043, exist_ok=False, line_thickness=10, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.9.16 torch-2.0.1+cu117 CUDA:0 (NVIDIA T600 Laptop GPU, 4096MiB)

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients
image 1/31 /home/hsack/practical_yolov5/datasets/custom_dataset/images/initial_tests/P10747.png: 448x640 1 clock, 68.9ms
image 2/31 /home/hsack/practical_yolov5/datasets/cu

In [16]:
# Run the model with the yolov5x weights
weights_folder = ""
weights = "yolov5x.pt"
run_name = "weights_"+ weights + "_" + datetime.now().strftime("%d-%m-%Y_%H%M%S")

weights_path = os.path.join(weights_folder, weights)
path_to_initial_test_folder = "./datasets/custom_dataset/images/initial_tests"

y5x_weights_run = f"./yolov5/runs/detect/{run_name}"

# detect with original yolov5s.pt weights
!python ./yolov5/detect.py --weights {weights} --img 640 --conf 0.25 --source {path_to_initial_test_folder} --name {run_name}  --line-thickness 10 --save-txt
      

/bin/bash: /home/hsack/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
detect: weights=['yolov5x.pt'], source=./datasets/custom_dataset/images/initial_tests, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=weights_yolov5x.pt_29-05-2023_185206, exist_ok=False, line_thickness=10, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-172-gc3c1304 Python-3.9.16 torch-2.0.1+cu117 CUDA:0 (NVIDIA T600 Laptop GPU, 4096MiB)

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
image 1/31 /home/hsack/practical_yolov5/datasets/custom_dataset/images/initial_tests/P10747.png: 448x640 2 cars, 88.4ms
image 2/31 /home/hsack/practical_yolov5/datasets/cus

In [1]:

print(f"The s model detected {countObjectsDetected(os.path.join(y5s_weights_run, 'labels'))}\t\tobjects")
print(f"The m model detected {countObjectsDetected(os.path.join(y5m_weights_run, 'labels'))}\tobjects")
print(f"The l model detected {countObjectsDetected(os.path.join(y5l_weights_run, 'labels'))}\t\tobjects")
print(f"The x model detected {countObjectsDetected(os.path.join(y5x_weights_run, 'labels'))}\tobjects")



NameError: name 'countObjectsDetected' is not defined

### Train the model

In [21]:
from datetime import datetime
weights = "yolov5m.pt" #Continue working the the m model 
train1_run_name = "train_"+ weights + "_" + datetime.now().strftime("%d-%m-%Y_%H%M%S")


# Run output in terminal 
# make sure you are in the 'yolov5' folder
print(f"python ./yolov5/train.py --img 640 --batch 16 --epochs 10 --data ./datasets/custom_dataset.yaml --weights {weights} --cache --name {train1_run_name}")

python ./yolov5/train.py --img 640 --batch 16 --epochs 10 --data ./datasets/custom_dataset.yaml --weights yolov5m.pt --cache --name train_yolov5m.pt_29-05-2023_185530
